In [1]:
import numpy as np
import random
import optuna
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, balanced_accuracy_score
import json
import os
from optuna.visualization import plot_optimization_history

/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 1234
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"

In [3]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)

In [4]:
def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(y_df["is_changed_trend_test"][:-1], y_df["is_changed_trend_predict"][:-1], digits=4)

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

In [5]:
# Cargar datos
train = pd.read_csv("../../../data/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/validation_set.csv", parse_dates=["date"])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values

In [6]:
def objective(trial):
    set_seeds(SEED)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 16)
    max_iter = trial.suggest_int("max_iter", 100, 500)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    l2_regularization = trial.suggest_float("l2_regularization", 0.0, 2.0)
    max_bins = trial.suggest_int("max_bins", 128, 255)

    model = HistGradientBoostingClassifier(
        learning_rate=learning_rate,
        max_depth=max_depth,
        max_iter=max_iter,
        min_samples_leaf=min_samples_leaf,
        l2_regularization=l2_regularization,
        max_bins=max_bins,
        random_state=SEED
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = trend_changes_true(y_val, y_pred)
    return score

In [ ]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=100)

[I 2025-08-20 20:08:24,376] A new study created in memory with name: no-name-75eaa248-0e76-4066-92a2-09d7544b5929
[I 2025-08-20 20:08:25,571] Trial 0 finished with value: 0.13636363636363635 and parameters: {'learning_rate': 0.01774894524799066, 'max_depth': 11, 'max_iter': 275, 'min_samples_leaf': 16, 'l2_regularization': 1.559951616237607, 'max_bins': 162}. Best is trial 0 with value: 0.13636363636363635.
[I 2025-08-20 20:08:27,480] Trial 1 finished with value: 0.13636363636363635 and parameters: {'learning_rate': 0.022892239910869177, 'max_depth': 14, 'max_iter': 484, 'min_samples_leaf': 18, 'l2_regularization': 0.7156345399157333, 'max_bins': 192}. Best is trial 0 with value: 0.13636363636363635.
[I 2025-08-20 20:08:28,771] Trial 2 finished with value: 0.2916666666666667 and parameters: {'learning_rate': 0.07748288441159408, 'max_depth': 12, 'max_iter': 248, 'min_samples_leaf': 12, 'l2_regularization': 1.0061663306156194, 'max_bins': 129}. Best is trial 2 with value: 0.291666666666

In [8]:
# Visualizar el historial de optimización
plot_optimization_history(study)

In [9]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de trend change: {study.best_value:.4f}")
best_params = study.best_params

Mejores hiperparámetros encontrados:
{'learning_rate': 0.11852258842059535, 'max_depth': 4, 'max_iter': 443, 'min_samples_leaf': 10, 'l2_regularization': 1.4427582668352443, 'max_bins': 182}
Mejor score de trend change: 0.3404


In [10]:
# Guardar los mejores hiperparámetros y su valor
history = []
if os.path.exists("best_hyperparams.json"):
    try:
        with open("best_hyperparams.json", "r") as f:
            history = json.load(f)
    except (json.JSONDecodeError, ValueError):
        history = []

# Guardar ambos en un solo diccionario
history.append({
    "params": study.best_params,
    "value": study.best_value
})
# Guardar el historial de hiperparámetros
with open("best_hyperparams.json", "w") as f:
    json.dump(history, f, indent=2)

In [11]:
# # Cargar historial de hiperparámetros y valores
# import json

# with open("best_hyperparams.json", "r") as f:
#     history = json.load(f)

# # Escoger el último (más reciente)
# best_params = history[-1]["params"]
# best_value = history[-1]["value"]

# # Si quieres ver todos:
# for i, entry in enumerate(history):
#     print(f"Hiperparámetros #{i+1}: {entry['params']}, Valor: {entry['value']}")

# # Si quieres escoger uno específico (por índice):
# # best_params = history[indice_que_quieras]["params"]
# # best_value = history[indice_que_quieras]["value"]

In [12]:
# Entrenar modelo final con los mejores hiperparámetros
set_seeds(SEED)
final_model = HistGradientBoostingClassifier(
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    max_iter=best_params["max_iter"],
    min_samples_leaf=best_params["min_samples_leaf"],
    l2_regularization=best_params["l2_regularization"],
    max_bins=best_params["max_bins"],
    random_state=SEED
)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_val)
print("\nHistGradientBoostingClassifier Trend Changes Score:\n", trend_changes_score(y_val, y_pred))


HistGradientBoostingClassifier Trend Changes Score:
               precision    recall  f1-score   support

       False     0.9091    0.8036    0.8531       112
        True     0.2667    0.4706    0.3404        17

    accuracy                         0.7597       129
   macro avg     0.5879    0.6371    0.5968       129
weighted avg     0.8244    0.7597    0.7855       129



In [13]:
# Reporte completo: precisión, recall y F1 por clase
report = classification_report(y_val, y_pred, digits=4)
print("Gradient Boosting Classifier Report:\n", report)
print("Balanced accuracy:", balanced_accuracy_score(y_val, y_pred))

Gradient Boosting Classifier Report:
               precision    recall  f1-score   support

          -1     0.8148    0.9167    0.8627        24
           0     0.3913    0.5000    0.4390        18
           1     0.9375    0.8523    0.8929        88

    accuracy                         0.8154       130
   macro avg     0.7145    0.7563    0.7315       130
weighted avg     0.8392    0.8154    0.8245       130

Balanced accuracy: 0.7563131313131312


In [14]:
#Versión más rápida y escalable: HistGradientBoostingClassifier
set_seeds(SEED)
model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_depth=6,          # profundidades algo mayores suelen ir bien aquí
    max_iter=400,         # equivalente a n_estimators
    random_state=SEED
)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print("\nHistGradientBoostingClassifier trend_changes_score:\n",
    trend_changes_score(y_val, y_pred))


HistGradientBoostingClassifier trend_changes_score:
               precision    recall  f1-score   support

       False     0.8878    0.7768    0.8286       112
        True     0.1935    0.3529    0.2500        17

    accuracy                         0.7209       129
   macro avg     0.5407    0.5649    0.5393       129
weighted avg     0.7963    0.7209    0.7523       129

